In [1]:
# import modules
import hvplot.pandas
import pandas as pd
import requests

from api_keys import geoapify_key

In [2]:
# creates the dataframe from the csv created
city_data_df = pd.read_csv("cities_dr.csv")
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,9.81,66,0,8.23,AR,1701992274
1,1,taunggyi,20.7833,97.0333,14.32,98,52,2.08,MM,1701992499
2,2,mpanda,-6.3655,31.0409,19.02,96,64,0.54,TZ,1701992500
3,3,badger,64.8000,-147.5333,-18.05,76,75,2.06,US,1701992500
4,4,fortuna,40.5982,-124.1573,11.07,92,75,3.09,US,1701992463


In [3]:
# creates the map based on humidity of cities
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    frame_width = 1000,
    frame_height = 600    
)

humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [4]:
# creates the hotel dataframe based on ideal weather conditions of cities
hotel_df = pd.DataFrame(city_data_df)
hotel_df["Hotel Name"] = ''
hotel_df = hotel_df[(hotel_df["Max Temp"] > 22) & (hotel_df["Max Temp"] < 30)]
hotel_df = hotel_df[hotel_df["Wind Speed"] < 5]
hotel_df = hotel_df[hotel_df["Cloudiness"] < 20]
hotel_df

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
28,28,port mathurin,-19.6833,63.4167,25.78,76,2,3.50,MU,1701992503,
39,39,san pedro pochutla,15.7333,-96.4667,25.18,71,4,2.08,MX,1701992504,
58,58,yeed,4.5500,43.0333,26.03,42,6,3.25,SO,1701992507,
66,66,antonio enes,-16.2325,39.9086,24.36,81,17,2.87,MZ,1701992508,
96,96,cabo san lucas,22.8909,-109.9124,28.17,41,0,4.12,MX,1701992283,
106,106,escuinapa,22.8500,-105.8000,25.55,65,0,2.07,MX,1701992514,
118,118,salalah,17.0151,54.0924,22.05,43,0,2.57,OM,1701992516,
123,123,kerikeri,-35.2268,173.9474,22.73,69,7,0.45,NZ,1701992517,
137,137,sassandra,4.9500,-6.0833,25.80,85,15,2.79,CI,1701992519,
160,160,mermoz boabab,14.7065,-17.4758,24.88,83,0,2.06,SN,1701992522,


In [5]:
# reads and uses API to gather information from the web to convert to JSON file and then fills the "Hotel Name" column in dataframe
radius = 10000
city = hotel_df["City"]
print("Starting hotel search")

for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    base_url = "https://api.geoapify.com/v2/places"
    params = {
        "apiKey":geoapify_key,
        "latitude": latitude,
        "longitude": longitude,
        "radius": radius,
        "limit": 20,
        "categories": "accommodation.hotel" 
} 
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   # hotel_url = base_url + "lat=" + str(latitude) + "&lon=" + str(longitude) + "&format=json&apiKey=" + str(geoapify_key)
    
    name_address = requests.get(base_url, params = params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

hotel_df

Starting hotel search
port mathurin - nearest hotel: Escale Vacances
san pedro pochutla - nearest hotel: Olimpia
yeed - nearest hotel: No hotel found
antonio enes - nearest hotel: Hotel Quirimbas
cabo san lucas - nearest hotel: Comfort Rooms
escuinapa - nearest hotel: Hotel Iq
salalah - nearest hotel: Muscat International Hotel
kerikeri - nearest hotel: Avalon Resort
sassandra - nearest hotel: Hotel Le Pollet
mermoz boabab - nearest hotel: La maison blanche
san jose de jachal - nearest hotel: San Martin
fortuna foothills - nearest hotel: Comfort Inn
saint croix - nearest hotel: The Buccaneer
altar - nearest hotel: Hotel Altar Inn
east ballina - nearest hotel: Ballina Beach Resort
januaria - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
byron bay - nearest hotel: Lord Byron Resort
woolgoolga - nearest hotel: Darlington Beach Resort
nicoya - nearest hotel: Hotel marianela
bonthe - nearest hotel: No hotel found
kismayo - nearest hotel: Kismayo Hotel
yauco

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
28,28,port mathurin,-19.6833,63.4167,25.78,76,2,3.50,MU,1701992503,Escale Vacances
39,39,san pedro pochutla,15.7333,-96.4667,25.18,71,4,2.08,MX,1701992504,Olimpia
58,58,yeed,4.5500,43.0333,26.03,42,6,3.25,SO,1701992507,No hotel found
66,66,antonio enes,-16.2325,39.9086,24.36,81,17,2.87,MZ,1701992508,Hotel Quirimbas
96,96,cabo san lucas,22.8909,-109.9124,28.17,41,0,4.12,MX,1701992283,Comfort Rooms
106,106,escuinapa,22.8500,-105.8000,25.55,65,0,2.07,MX,1701992514,Hotel Iq
118,118,salalah,17.0151,54.0924,22.05,43,0,2.57,OM,1701992516,Muscat International Hotel
123,123,kerikeri,-35.2268,173.9474,22.73,69,7,0.45,NZ,1701992517,Avalon Resort
137,137,sassandra,4.9500,-6.0833,25.80,85,15,2.79,CI,1701992519,Hotel Le Pollet
160,160,mermoz boabab,14.7065,-17.4758,24.88,83,0,2.06,SN,1701992522,La maison blanche


In [22]:
# generates map based with hotel name and country in hover message
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = "City",
    size = "Humidity",
    frame_width =1000,
    frame_height =700,
    hover_cols=["Hotel Name", "Country"]
)


hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)